In [ ]:
import yaml
import add_packages
from pprint import pprint
import os, re
import pandas as pd
# import tqdm
from tqdm.auto import tqdm

from my_langchain import (
  document_loaders, text_splitters, text_embedding_models, vectorstores, 
  chat_models, prompts, utils, output_parsers, agents, documents, llms,
  runnables
)

tqdm.pandas(desc="Processing")

with open("../my_configs/vtc.yaml", 'r') as file:
    configs_vtc = yaml.safe_load(file)

# Data

## Lectures Content

### Combine

In [ ]:
def extract_course_name(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        first_line = file.readline().strip()
        return first_line[len("# Course: "):]

In [ ]:
def remove_extra_newlines(text):
    while '\n\n' in text:
        text = text.replace('\n\n', '\n')
    return text


def remove_double_spaces(text):
    while '  ' in text:
        text = text.replace('  ', ' ')
    return text


def process_add_space_after_hash(text):
    pattern = r'##(\w)'
    processed_text = re.sub(pattern, r'## \1', text)
    return processed_text


def process_remove_quotes(text):
    cleaned_text = text.replace('"', '')
    return cleaned_text


def process_text_file(input_file, output_file, functions):
    # Read the first line of the input file
    with open(input_file, 'r') as file:
        first_line = file.readline().strip()
        len_first_line = len(first_line)
    

    # Extract course name from filename
    course_name = os.path.splitext(os.path.basename(input_file))[0]
    # If the first line doesn't match the filename, replace it with the filename's course name
    first_line = "# Course: " + course_name

    # Reopen the file to process the entire content
    with open(input_file, 'r') as file:
        text = file.read()

    # Apply processing functions
    for func in functions:
        text = func(text)

    # Prepend the modified first line to the processed text
    text = first_line + '\n' + text[len_first_line + 1:]

    # Write the processed text to the output file
    with open(output_file, 'w') as file:
        file.write(text)


# Define the list of functions to apply
functions_to_apply = [
    process_remove_quotes,
    remove_extra_newlines,
    remove_double_spaces,
    process_add_space_after_hash,
]


In [ ]:
# # Example usage:
# input_file_path = '../data/vtc1/script/KỸ NĂNG TÌM VIỆC LÀM THÊM.txt'
# output_file_path = '../data/vtc1/script/KỸ NĂNG TÌM VIỆC LÀM THÊM.txt'
# process_text_file(input_file_path, output_file_path, functions_to_apply)

### Process files

In [ ]:
# # Specify the folder containing the files
# folder_path = '../data/vtc2/script/'

# # Loop through each file in the folder
# for filename in os.listdir(folder_path):
#     if filename.endswith('.txt'):  # Process only text files
#         input_file_path = os.path.join(folder_path, filename)
#         output_file_path = os.path.join(
#             folder_path, filename)  # Output file path
#         process_text_file(input_file_path, output_file_path,
#                           functions_to_apply)

### Create documents 

#### ONE course


In [ ]:
# # Example usage:
# file_path = "../data/vtc1/script/HỘI HỌA - MÀU SẮC VÀ PHỐI MÀU CƠ BẢN.txt"
# course_name = extract_course_name(file_path).lower()

# text_loader_lectures_content = document_loaders.TextLoader(file_path)
# document = text_loader_lectures_content.load()

# text_splitter = text_splitters.RecursiveCharacterTextSplitter(
#   chunk_size=1000, chunk_overlap=200,
# )
# docs_lectures_content = text_splitter.split_documents(document)

# metadatas = {
#   "data": "lectures content",
#   "course_name": course_name,
# }
# utils.remove_metadata(docs_lectures_content, "source")
# utils.update_metadata(docs_lectures_content, metadatas)

#### MULTIPLE course


In [ ]:

def process_folder(folder_path):
    data_dict = {}
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith('.txt'):
                file_path = os.path.join(root, file_name)
                course_name = extract_course_name(file_path).lower()

                text_loader_lectures_content = document_loaders.TextLoader(file_path)
                document = text_loader_lectures_content.load()

                text_splitter = text_splitters.RecursiveCharacterTextSplitter(
                    chunk_size=1000, chunk_overlap=200,
                )
                docs_lectures_content = text_splitter.split_documents(document)

                metadatas = {
                    "data": "lectures content",
                    "course_name": course_name,
                }
                utils.remove_metadata(docs_lectures_content, "source")
                utils.update_metadata(docs_lectures_content, metadatas)

                data_dict[course_name] = docs_lectures_content

    return data_dict


folder_path = "../data/vtc2/script/"
docs_lectures_content_dict = process_folder(folder_path)
for file_name, content in docs_lectures_content_dict.items():
    print(f"File: {file_name}, Course: {content[0].metadata['course_name']}")

## FAQ

In [ ]:
text_loader_faq = document_loaders.TextLoader(
  "../data/vtc/faq1.txt"
)
document = text_loader_faq.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
  # chunk_size=500, chunk_overlap=100,
  separators=["##"], chunk_size=150, chunk_overlap=0,
)
docs_faq = text_splitter.split_documents(document)
docs_faq = docs_faq[1:]
metadatas = {
  "data": "frequently asked questions"
}
utils.remove_metadata(docs_faq, "source")
utils.update_metadata(docs_faq, metadatas)

## Courses list

In [ ]:
model = chat_models.chat_openai

template1 = """\
Dựa trên mô tả khóa học, đưa ra chỉ nội dung cốt lõi sẽ được dạy của khóa học. Trả lời dưới dạng gạch đầu dòng.

Đây là mô tả khóa học:
{text}"""

template2 = """\
Delete bullet points. Replace line breaks with dots.
Example Input:
'- Kiến thức cơ bản về màu sắc\n- Sử dụng ánh sáng và nhiệt độ trong màu sắc\n- Nguyên lý phối màu\n- Thực hành tô màu với vòng tròn thuần sắc\n- Hướng dẫn bởi giảng viên chuyên ngành thiết kế'

Example Output:
'Kiến thức cơ bản về màu sắc. Sử dụng ánh sáng và nhiệt độ trong màu sắc. Nguyên lý phối màu. Thực hành tô màu với vòng tròn thuần sắc. Hướng dẫn bởi giảng viên chuyên ngành thiết kế'

Input:
{text}
"""

prompt_template1 = prompts.PromptTemplate.from_template(template1)
prompt_template2 = prompts.PromptTemplate.from_template(template2)

chain1 = prompt_template1 | model | output_parsers.StrOutputParser()
chain2 = prompt_template2 | model | output_parsers.StrOutputParser()

chain = runnables.RunnablePassthrough.assign(
  text=chain1
).assign(
  text=chain2
)

def process_course_description(text: str) -> str:
  result = chain.invoke({"text": text})['text']
  return result

query = 'Bạn yêu thích việc sử dụng màu sắc, nhưng lại không biết làm sao để phối ra những màu sắc mà bản thân mong muốn. Vậy thì khóa học "Màu sắc và cách phối màu cơ bản" sẽ mang đến cho bạn những kiến thức cơ bản về màu sắc như việc sử dụng ánh sáng, nhiệt độ trong màu sắc hay thú vị hơn là nắm được các nguyên lý phối màu. Không những thế bạn còn có thể thực hành tô màu với vòng tròn thuần sắc cùng Mr. Anh Thi - Giảng viên chuyên ngành thiết kế. Để có thể có những trải nghiệm tuyệt vời trên còn chần chờ gì mà không nhanh tay đăng ký khóa học để tiếp thu những kiến thức hữu ích bạn nhé!'
result = process_course_description(query)

pprint(result)

In [ ]:
path_courses_list = "../data/vtc2/courses_list1.csv"
courses_list_cols = utils.get_csv_column_names(path_courses_list)

csv_loader_courses_list = document_loaders.CSVLoader(
    path_courses_list,
    # source_column="No",
    csv_args={
        "delimiter": ",",
        # "quotechar": "''",
        "fieldnames": courses_list_cols,
    },
)
document = csv_loader_courses_list.load()[1:]
docs_courses_information = document

metadatas = {
    "data": "courses information"
}

utils.remove_metadata(docs_courses_information, "source")
utils.remove_metadata(docs_courses_information, "row")
utils.update_metadata(docs_courses_information, metadatas)

# Vector store 

## Lectures Content

In [ ]:
qdrant_lectures_content = vectorstores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs_vtc,
  **configs_vtc["vector_db"]["qdrant"]["lectures_content"],
)

### Add multiple courses


In [ ]:
# for docs in docs_lectures_content_dict.values():
#   qdrant_lectures_content.add_documents(docs)

#### If error

In [ ]:
# # Check position based on len(docs) of the course
# for position, (key, value) in enumerate(docs_lectures_content_dict.items()):
#     if len(value) == 71:
#         print("Key:", key)
#         print("Position:", position)

In [ ]:
# # Continue if error
# for docs in (list(docs_lectures_content_dict.values()))[36:]:
#   qdrant_lectures_content.add_documents(docs)

### Add one course


In [ ]:
# qdrant_lectures_content.add_documents(docs_lectures_content)

## FAQ

In [ ]:
qdrant_faq = vectorstores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs_vtc,
  **configs_vtc["vector_db"]["qdrant"]["faq"]
)

In [ ]:
# qdrant_faq.add_documents(docs_faq)

## Courses List

In [ ]:
qdrant_courses_information = vectorstores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs_vtc,
  **configs_vtc["vector_db"]["qdrant"]["courses_information"]
)

In [ ]:
# qdrant_courses_information.add_documents(docs_courses_information)

#### ONE course


# Tools

In [ ]:
system_message_onlinica = configs_vtc["prompts"]["system_message_onlinica"]
prompt_onlinica = prompts.create_prompt_custom_agent_openai_tools(system_message_onlinica)

In [ ]:
tools = [
  qdrant_lectures_content.retriever_tool,
  qdrant_faq.retriever_tool,
  qdrant_courses_information.retriever_tool,
]

llm = chat_models.chat_openai
agent = agents.MyAgent(
  prompt=prompt_onlinica, tools=tools,
  agent_type=configs_vtc["agents"]["agent_type_onlinica"], 
  llm=llm
)

In [ ]:
questions = [
  "xin chào. Tên tôi là Bob.",
  "bạn có nhớ tên tôi là gì không",
  
  "digital marketing là gì",
  
  "làm cách nào để đăng ký tài khoản onlinica",
  "có mấy loại tài khoản onlinica",
  "các khoá học tại onlinica có thời hạn sử dụng bao lâu",
  "onlinica có mấy hình thức thanh toán",
  "có thể thanh toán bằng momo được không",
  
  "các khóa học về design",
  "các khóa học về trí tuệ nhân tạo",
  "các khóa học về  ai",
  "các khóa học của nguyễn ngọc tú uyên",
  "các khóa học của tú uyên",
  "các khóa học thầy trần anh tuấn dạy",
  
  "cách quản lý thời gian",
  "nguyên lý phối màu",
]


In [ ]:
input_message = "nguyên lý phối màu"

# await agent.invoke_agent_stream(questions[2])
await agent.invoke_agent_stream(input_message)

In [ ]:
await agent.invoke_agent_stream("Chi tiết hơn")

# Test Anthropic

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_xml_agent
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults

# tools = [TavilySearchResults(max_results=1)]
tools = [
    qdrant_lectures_content.retriever_tool,
    qdrant_faq.retriever_tool,
    qdrant_courses_information.retriever_tool,
]

# Get the prompt to use - you can modify this!
# prompt = prompts.create_prompt_custom_agent_xml_tools(system_message_onlinica)
prompt = prompts.create_prompt_custom_agent_xml_tools()

# Choose the LLM that will drive the agent
llm = ChatAnthropic(model="claude-3-haiku-20240307")

# Construct the XML agent
agent = create_xml_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "các khóa học thầy trần anh tuấn"})['output']
pprint(result)